In [1]:
# generate sample data
import pandas as pd
import random


# generate sample data
data = {
    'roll_num': range(1, 1000001),
    'name': [f'student {i}' for i in range(1, 1000001)],
    'sub1': [round(random.uniform(600, 1000), 2) for _ in range(1000000)],
    'sub2': [round(random.uniform(600, 1000), 2) for _ in range(1000000)],
    'sub3': [round(random.uniform(600, 1000), 2) for _ in range(1000000)],
    'sub4': [round(random.uniform(600, 1000), 2) for _ in range(1000000)],
    'sub5': [round(random.uniform(600, 1000), 2) for _ in range(1000000)],
    'sub6': [round(random.uniform(600, 1000), 2) for _ in range(1000000)],
    'sub7': [round(random.uniform(600, 1000), 2) for _ in range(1000000)]
}


In [2]:
# Create a dataframe
df = pd.DataFrame(data)

In [3]:
# Save to CSV
df.to_csv('students.csv', index=False)
print("Sample CSV file 'students.csv' created!")

Sample CSV file 'students.csv' created!


In [4]:
# ETL pipeline implementation

import duckdb
import pandas as pd
import time

# Extract: load time to CSV
start_time = time.time()
df = pd.read_csv('students.csv')
extract_time = time.time() - start_time
print(f"Data extracted in {extract_time:.2f} seconds.")

Data extracted in 0.55 seconds.


In [5]:
df.head()

,roll_num,name,sub1,sub2,sub3,sub4,sub5,sub6,sub7
0,1,student 1,616.79,971.42,681.52,708.81,881.05,822.41,850.59
1,2,student 2,800.10,922.32,985.76,992.82,841.45,922.07,946.11
2,3,student 3,664.89,677.54,705.38,628.24,925.87,736.38,638.88
3,4,student 4,960.66,862.70,643.17,667.93,820.62,896.77,948.53
4,5,student 5,698.17,696.24,884.86,745.53,610.06,659.08,705.93


In [6]:
# Connect to duck DB
conn = duckdb.connect(database=':memory:')

In [7]:
# Transform: clean and aggregate data
start_time = time.time()
df['average_score'] = df [['sub1', 'sub2', 'sub3', 'sub4', 'sub5', 'sub6', 'sub7']].mean(axis=1)
transform_time = time.time() - start_time
print(f"Data transformed in {transform_time:.2f} seconds.")

Data transformed in 0.08 seconds.


In [8]:
# load data into DuckDB table
start_time = time.time()
conn.execute("CREATE TABLE students AS SELECT * FROM df")
load_time = time.time() - start_time
print(f"Data loaded into DuckDB in {load_time:.2f} seconds.")

Data loaded into DuckDB in 0.18 seconds.


In [9]:
# performance stats
total_time = extract_time + transform_time + load_time
print(f"Total ETL time: {total_time:.2f} seconds.")

Total ETL time: 0.81 seconds.


In [10]:
# Analyze: Querying the data
start_time = time.time()
result = conn.execute("SELECT * FROM students WHERE average_score > 850").fetchall()
query_time = time.time() - start_time
print(f"Data queried in {query_time:.2f} seconds")

Data queried in 0.17 seconds


In [11]:
# Output results
print("Query Result:")
for row in result[:10]:  # Display first 10 results for brevity
    print(row)

Query Result:
(2, 'student 2', 800.1, 922.32, 985.76, 992.82, 841.45, 922.07, 946.11, 915.8042857142857)
(10, 'student 10', 938.3, 705.44, 861.62, 913.12, 991.64, 986.75, 852.19, 892.722857142857)
(40, 'student 40', 925.2, 758.75, 958.98, 812.36, 820.68, 939.02, 999.02, 887.7157142857143)
(42, 'student 42', 999.03, 955.24, 894.15, 662.87, 811.9, 924.44, 896.1, 877.6757142857142)
(43, 'student 43', 633.32, 979.44, 922.28, 863.02, 969.71, 894.77, 769.71, 861.7500000000001)
(50, 'student 50', 971.5, 715.75, 953.28, 732.43, 951.79, 909.97, 870.92, 872.2342857142858)
(64, 'student 64', 807.6, 883.31, 893.11, 915.23, 936.11, 808.48, 789.59, 861.9185714285715)
(67, 'student 67', 842.56, 809.57, 919.16, 857.29, 821.4, 859.81, 973.2, 868.9985714285713)
(73, 'student 73', 982.65, 939.42, 806.35, 780.68, 629.72, 841.77, 982.15, 851.8199999999999)
(77, 'student 77', 666.71, 807.47, 928.29, 898.91, 970.23, 996.43, 702.69, 852.9614285714288)


In [12]:
# Summary of performance stats
performance_stats = {
    'Extract Time': extract_time,
    'Transform Time': transform_time,
    'Load Time': load_time,
    'Query Time': query_time,
    'Total ETL Time': total_time
}


In [13]:
print("\nPerformance Stats:")
for key, value in performance_stats.items():
    print(f"{key}: {value:.2f} seconds")


Performance Stats:
Extract Time: 0.55 seconds
Transform Time: 0.08 seconds
Load Time: 0.18 seconds
Query Time: 0.17 seconds
Total ETL Time: 0.81 seconds
